In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Import function for calculating standard error
from scipy.stats import sem

# Import statistical tests
from scipy.stats import shapiro # Shapiro-Wilk test to check normality
from scipy.stats import levene # Levene's test instead of Barlett's test to check homogeneity of variances of groups that might not be normally distributed
from scipy.stats import kruskal # Kruskal-Wallis test used to determine whether or not there is a statistically significant difference between the medians of three or more independent groups. It is considered to be the non-parametric equivalent of the One-Way ANOVA.
from scipy.stats import f_oneway # One-way ANOVA test to compare multiple groups of data that is normally distributed

# Install scikit-posthocs package
%pip install scikit-posthocs

# Import post-hoc test for multiple comparisons
from scikit_posthocs import posthoc_dunn # Dunn's test to make multiple comparisons between groups

In [ ]:
# Load dataset
BD = pd.read_excel('BD Maestría WT.xlsx', sheet_name='full_data') # The full data is stored in the full_data sheet from the file so we select it and store it on the variable BD

In [ ]:
# Function to avoid overlapping when making the swarmplot over the barplot
def simple_beeswarm(y, nbins=None):
    """
    Returns x coordinates for the points in ``y``, so that plotting ``x`` and
    ``y`` results in a bee swarm plot.
    """
    y = np.asarray(y)
    if nbins is None:
        nbins = len(y) // 6

    # Get upper bounds of bins
    x = np.zeros(len(y))
    ylo = np.min(y)
    yhi = np.max(y)
    dy = (yhi - ylo) / nbins
    ybins = np.linspace(ylo + dy, yhi - dy, nbins - 1)

    # Divide indices into bins
    i = np.arange(len(y))
    ibs = [0] * nbins
    ybs = [0] * nbins
    nmax = 0
    for j, ybin in enumerate(ybins):
        f = y <= ybin
        ibs[j], ybs[j] = i[f], y[f]
        nmax = max(nmax, len(ibs[j]))
        f = ~f
        i, y = i[f], y[f]
    ibs[-1], ybs[-1] = i, y
    nmax = max(nmax, len(ibs[-1]))

    # Assign x indices
    if nmax//2 > 0:
      dx = 1 / (nmax // 2)
    else:
      dx = 1
    for i, y in zip(ibs, ybs):
        if len(i) > 1:
            j = len(i) % 2
            i = i[np.argsort(y)]
            a = i[j::2]
            b = i[j+1::2]
            x[a] = (0.5 + j / 3 + np.arange(len(b))) * dx
            x[b] = (0.5 + j / 3 + np.arange(len(b))) * -dx

    return x

In [ ]:
# Function to make a swarmplot over a barplot
def graficar(Variable):
  s = ['"machos"', '"hembras"'] # Plot for both genders one beside the other
  simbolo = ["♂", "♀"]
  max_ylim = []
  fig, axs =plt.subplots(1, 2, figsize=(15,5)) # Create the space for both plots
  for j in [0,1]:
    y = (BD.query(f'Dieta == "CD" and Edad == "Y" and Sexo == {s[j]}')[Variable],
      BD.query(f'Dieta == "KD" and Edad == "Y" and Sexo == {s[j]}')[Variable],
      BD.query(f'Dieta == "CD" and Edad == "O" and Sexo == {s[j]}')[Variable],
      BD.query(f'Dieta == "KD" and Edad == "O" and Sexo == {s[j]}')[Variable]) # Coordinates for y need to be searched for in order CD_Y, KD_Y, CD_O, KD_O
    x = [0, 1, 2, 3] # x-coordinates for each bar

    colors = np.asarray([(0, 53, 63, 150), (212, 111, 77, 150), (30, 169, 188, 150), (255, 191, 102, 150)]) # Initialize an array with RGB colors
    colors = [i/255 for i in colors]    # Corresponding colors mapped to values from 0 to 1 instead of 0 to 255 so that Python interprets them correctly

    w = 0.8 # Bar width

    # Plotting the barplot
    axs[j].bar(x,
          height=[np.mean(yi) for yi in y], # Height of each bar is a value form the data extracted to y
          yerr=[sem(yi) for yi in y],    # Error bars
          capsize=6, # Error bar's cap width in points
          width=w,    # Bar width
          color = (0,0,0,0),  # Face color transparent
          edgecolor = colors, # Edge colors
          ecolor = (0.65, 0.65, 0.65, 0.5),    # Error bar colors
          tick_label = ['CD Y', 'KD Y', 'CD O', 'KD O'] # Labels for each bar
          )
    # Removing the frame
    axs[j].spines['top'].set_visible(False)
    axs[j].spines['right'].set_visible(False)

    # Seting the titles for the plot based on a portion of the title in the corresponding colum in the excel file
    if len(Variable.split(" ")) > 1:
      axs[j].set_title(f'{Variable.split(" ")[0]}  {simbolo[j]}  -  Semana {Variable.split(" ")[-1]}\n\n\n\n')
      if Variable.split(" ")[0] == "%":
        axs[j].set_title(f'% Cambio de peso {simbolo[j]}  -  Semana {Variable.split(" ")[-1]}\n\n\n\n')
    else:
      axs[j].set_title(f'{Variable} {simbolo[j]}\n\n\n\n')

    # Initialize an array of markers to identify each dot form the swarm plot
    markers = ['$0$', '$1$', '$2$', '$3$', '$4$', '$5$', '$6$', '$7$', '$8$', '$9$']

    # Plotting the swarm plot on top of the barplot
    for i in x:
      for a, b in enumerate(y[i]):
          axs[j].scatter(x[i]+simple_beeswarm(y[i], nbins=10)[a]*w/2,
                    b,
                    color=colors[i],
                    marker = markers[a]
                    )


    # Changing the lables for each type of data ploted
    label=""
    if Variable.split(" ")[0] == "Peso":
      label = "Peso (g)\n"
      axs[j].set_ylim(20,50)
    if Variable.split(" ")[0] == "%":
      label = "Cambio de peso (%)\n"
      axs[j].set_ylim(-30,34)
    if Variable.split(" ")[0] == "Glucosa":
      label = "Glucosa en sangre\n(mg/dL)"
      axs[j].set_ylim(65,200)
    if Variable.split(" ")[0] == "Cetónicos":
      label = "BHB en sangre \n (mmol/L)"
      axs[j].set_ylim(0,3.3)
    if Variable.split(" ")[0] == "Fragilidad":
      label = "Índice de fragilidad\n"
      axs[j].set_ylim(-0.01,0.23)
    if Variable.split(" ")[0] == "Kondziela":
      label = "Tiempo (s) / peso (g)\n"
      axs[j].set_ylim(-0.1,2.8)
    if Variable.split(" ")[0] == "Triglicéridos":
      label = "Triglicéridos en sangre\n(mg/dL)"
      axs[j].set_ylim(0,1.25)
    if Variable.split(" ")[0] == "Colesterol":
      label = "Colesterol en sangre\n(mg/dL)"
      axs[j].set_ylim(0,1.55)
  axs[1].set_ylabel("")
  axs[0].set_ylabel(label)
  plt.show()

# Testing for an individual plot
graficar("Triglicéridos")

In [ ]:
# Functions to test for normality, equality of variances, and differences between groups

# Function to extract a group from a portion of the complete dataset
def sub_grupo (Dieta, Edad, Sexo, p):
  Grupo = BD[(BD["Sexo"] == Sexo) & (BD["Edad"] == Edad) & (BD["Dieta"] == Dieta)][p]
  return(Grupo)

# Normality test
def normality_test (grupo):
  if shapiro(grupo)[1] < 0.05:
    answer = "0"
  else:
    answer = "1"
  return answer

# Equality of variances test
def eq_variances_test (grupo1, grupo2, grupo3, grupo4):
  if levene(grupo1, grupo2, grupo3, grupo4)[1] < 0.05:
    hsd = "0"
  else:
    hsd = "1"
  return hsd

# Kruskal Wallis test
def kruskal_test (grupo1, grupo2, grupo3, grupo4):
  if kruskal(grupo1, grupo2, grupo3, grupo4, nan_policy = "omit")[1] < 0.05:
    diff = ""
  else:
    diff = "Iguales"
  return diff

# ANOVA test
def anova_test (grupo1, grupo2, grupo3, grupo4):
  if f_oneway(grupo1, grupo2, grupo3, grupo4)[1] < 0.05:
    diff = ""
  else:
    diff = "Iguales"
  return diff

# Dunn post-hoc test
def dunn_test (grupo):
  print("Dunn test")
  for a,b in enumerate (["CD-Y", "KD-Y", "CD-O", "KD-O"]):
      for c, d in enumerate (["CD-Y", "KD-Y", "CD-O", "KD-O"]):
        if (a+1) < (c+1):
          if posthoc_dunn(grupo, p_adjust = 'fdr_bh')[a+1][c+1] < 0.05:
            print(b, " vs ", d, " p-value: ", posthoc_dunn(grupo, p_adjust = 'fdr_bh')[a+1][c+1] )


In [ ]:
# Loop to plot every variable and make all statistical tests to each of them
for (a,b) in enumerate(BD):
  if a > 6:
    print("\n" + "\n" + b)
    if b != "% Cambio Peso 0":
      graficar(b)
      for s in ["machos", "hembras"]:
        normality_list = []
        grouped_list = []
        print("\n" + s)
        for e in ["Y", "O"]:
          for d in ["CD", "KD"]:
            grupo = sub_grupo(d, e, s, b)
            normality_list += [normality_test(grupo)]
            grouped_list += [grupo]
        print("normality - ", normality_list)
        print("variance test - ", eq_variances_test(*grouped_list))
        print("Kruskal test - ",  kruskal_test(*grouped_list))
        print("Anova test - ",  anova_test(*grouped_list))
        print(dunn_test(grouped_list))